In [1]:
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostRegressor
from algolab import API
from datetime import datetime
from config import *
import pandas as pd, numpy as np, json, os

import yfinance as yf

In [2]:
son=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=son.iloc[-1,:]
son=son.iloc[:-1,:]
hisseler=son["Kod"].values
hisseler=np.array(hisseler)
for i in range(len(hisseler)):
    hisseler[i]=hisseler[i]+".IS"

In [291]:
df=pd.DataFrame()
for hisse in hisseler:
    hisse_verileri = yf.download(hisse,start="2010-01-01")  # Örnek tarih aralığı
    hisse_kapanis = hisse_verileri[['High','Low','Open','Close','Volume']]
    hisse_kapanis["Hisse"]=hisse
    hisse_kapanis.loc[hisse_kapanis["Close"] > hisse_kapanis["High"], "High"] = hisse_kapanis["Close"]
    hisse_kapanis.loc[hisse_kapanis["Close"] < hisse_kapanis["Low"], "Low"] = hisse_kapanis["Close"]

    df=pd.concat([df,hisse_kapanis],axis=0)
df

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,High,Low,Open,Close,Volume,Hisse
Date,,,,,,
2023-06-15,27.500000,27.500000,27.500000,27.500000,262214.0,A1CAP.IS
2023-06-16,30.240000,30.240000,30.240000,30.240000,1169499.0,A1CAP.IS
2023-06-20,31.100000,29.940001,31.000000,29.940001,8064437.0,A1CAP.IS
2023-06-21,26.959999,26.959999,26.959999,26.959999,2147415.0,A1CAP.IS
2023-06-22,27.620001,25.500000,25.620001,25.940001,71898180.0,A1CAP.IS
...,...,...,...,...,...,...
2024-05-29,7.610000,7.220000,7.220000,7.390000,4984541.0,ZRGYO.IS
2024-05-30,7.970000,7.220000,7.590000,7.890000,11925383.0,ZRGYO.IS
2024-05-31,8.200000,7.530000,7.910000,8.160000,20301298.0,ZRGYO.IS


In [3]:
df=pd.read_csv("df.csv")
df=df.set_index(pd.to_datetime(df["Date"]))
del df["Date"]
df

,High,Low,Open,Close,Volume,Hisse
Date,,,,,,
2023-06-15,27.500000,27.500000,27.500000,27.500000,262214.0,A1CAP.IS
2023-06-16,30.240000,30.240000,30.240000,30.240000,1169499.0,A1CAP.IS
2023-06-20,31.100000,29.940001,31.000000,29.940001,8064437.0,A1CAP.IS
2023-06-21,26.959999,26.959999,26.959999,26.959999,2147415.0,A1CAP.IS
2023-06-22,27.620001,25.500000,25.620001,25.940001,71898180.0,A1CAP.IS
...,...,...,...,...,...,...
2024-05-29,7.610000,7.220000,7.220000,7.390000,4984541.0,ZRGYO.IS
2024-05-30,7.970000,7.220000,7.590000,7.890000,11925383.0,ZRGYO.IS
2024-05-31,8.200000,7.530000,7.910000,8.160000,20301298.0,ZRGYO.IS


In [4]:
data=df.copy()

In [5]:
df["Close"]=np.round(df["Close"],2)
df["High"]=np.round(df["High"],2)
df["Low"]=np.round(df["Low"],2)
df["Open"]=np.round(df["Open"],2)
df["Volume"]=df["Volume"]*df["Close"]
df.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse']

In [6]:
canlı=pd.read_excel("./excel/FiyatPenceresi.Xlsx")
endeks=canlı.iloc[-1,:]
canlı=canlı.iloc[:-1,:]
canlı=canlı[["Yks","Dşk","Açl.Gün","Son.Fyt","Hcm","Kod","Lot"]]
canlı.columns=['High', 'Low', 'Open', 'Adj Close', 'Volume', 'Hisse',"Lot"]
canlı.index = pd.to_datetime(['2024-05-30'] * len(canlı))
canlı = canlı[~canlı['Hisse'].isin(['IDEAS', 'ISATR', 'ISKUR', 'ITTFH'])]
canlı["Lot"]=canlı["Lot"].astype("string")
canlı

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-05-30,"28,28","27,38","27,48","27,86",30.033.253,A1CAP,1.077.426
2024-05-30,"145,40","140,20","140,40","144,00",18.915.874,ACSEL,131.965
2024-05-30,"671,00","648,50","663,00","662,00",160.002.438,ADEL,242.113
2024-05-30,"2,34","2,27","2,31","2,32",115.315.479,ADESE,50.003.224
2024-05-30,"27,22","25,74","27,08","25,74",41.929.646,ADGYO,1.578.582
...,...,...,...,...,...,...,...
2024-05-30,"6,73","6,39","6,60","6,43",31.645.690,YYAPI,4.847.454
2024-05-30,"12,50","12,20","12,35","12,24",90.523.574,YYLGD,7.343.653
2024-05-30,"11,74","11,02","11,04","11,12",47.109.826,ZEDUR,4.130.931
2024-05-30,"5,68","5,53","5,55","5,57",1.004.573.458,ZOREN,178.988.903


In [7]:
canlı=canlı.dropna()

In [8]:
for i in range(len(canlı)):
    if len(canlı.iloc[i, -1]) <= 7:
        if '.' in canlı.iloc[i, -1]:
            parts = canlı.iloc[i, -1].split('.')
            decimal_part = parts[1]
            if len(decimal_part) == 1:
                decimal_part += '00'
            elif len(decimal_part) == 2:
                decimal_part += '0'
            elif len(decimal_part) == 3:
                canlı.iloc[i, -1] = parts[0] + decimal_part[:2]
            canlı.iloc[i, -1] = parts[0] + decimal_part

In [9]:
kod=canlı["Hisse"].unique()
for i in range(len(kod)):
    kod[i]=kod[i]+".IS"

In [14]:
fark=np.setdiff1d(df["Hisse"].unique(),kod)

In [15]:
df = df[~df['Hisse'].isin(fark)]

In [18]:
canlı["Hisse"]=kod
canlı["High"]=canlı["High"].str.replace(",",".")
canlı["Low"]=canlı["Low"].str.replace(",",".")
canlı["Open"]=canlı["Open"].str.replace(",",".")
canlı["Adj Close"]=canlı["Adj Close"].str.replace(",",".")
canlı["Lot"]=canlı["Lot"].astype(str).str.replace(".", "")
canlı[["High","Low","Open","Adj Close","Lot"]]=canlı[["High","Low","Open","Adj Close","Lot"]].astype(float)
canlı["Volume"]=canlı["Lot"]*canlı["Adj Close"]
canlı

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\552829948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["Hisse"]=kod
C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\552829948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  canlı["High"]=canlı["High"].str.replace(",",".")
C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\552829948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,High,Low,Open,Adj Close,Volume,Hisse,Lot
2024-05-30,28.28,27.38,27.48,27.86,3.001709e+07,A1CAP.IS,1077426.0
2024-05-30,145.40,140.20,140.40,144.00,1.900296e+07,ACSEL.IS,131965.0
2024-05-30,671.00,648.50,663.00,662.00,1.602788e+08,ADEL.IS,242113.0
2024-05-30,2.34,2.27,2.31,2.32,1.160075e+08,ADESE.IS,50003224.0
2024-05-30,27.22,25.74,27.08,25.74,4.063270e+07,ADGYO.IS,1578582.0
...,...,...,...,...,...,...,...
2024-05-30,6.73,6.39,6.60,6.43,3.116913e+07,YYAPI.IS,4847454.0
2024-05-30,12.50,12.20,12.35,12.24,8.988631e+07,YYLGD.IS,7343653.0
2024-05-30,11.74,11.02,11.04,11.12,4.593595e+07,ZEDUR.IS,4130931.0
2024-05-30,5.68,5.53,5.55,5.57,9.969682e+08,ZOREN.IS,178988903.0


In [25]:
canlı=canlı[canlı["Hisse"].isin(df["Hisse"].loc["2024-06-04"])]

In [27]:
df["High"].loc["2024-06-04"]=canlı["High"].values
df["Low"].loc["2024-06-04"]=canlı["Low"].values
df["Open"].loc["2024-06-04"]=canlı["Open"].values
df["Adj Close"].loc["2024-06-04"]=canlı["Adj Close"].values
df["Volume"].loc["2024-06-04"]=canlı["Volume"].values

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1918069527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["High"].loc["2024-06-04"]=canlı["High"].values
C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1918069527.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the ass

In [28]:
df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,
                np.where(df["Adj Close"] < 50, 0.02,
                    np.where(df["Adj Close"] < 100, 0.05,
                        np.where(df["Adj Close"] < 250, 0.1,
                            np.where(df["Adj Close"] < 500, 0.25,
                                np.where(df["Adj Close"] < 1000, 0.5,
                                    np.where(df["Adj Close"] < 2500, 1, 2.5)
                                )  # Buradaki parantez eksikti.
                            )
                        )
                    )
                )
            )

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1526774384.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Kademe"] = np.where(df["Adj Close"] < 20, 0.01,


In [29]:
import math
def yuvarla_asagi(row):
    fiyat = row['Adj Close']
    kadem = row['Kademe']
    return math.floor(fiyat * 1.1 / kadem) * kadem

df["Tavan"]=df.apply(yuvarla_asagi, axis=1)
df["Tavan"]=np.round(df["Tavan"],2)

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1680495506.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=df.apply(yuvarla_asagi, axis=1)
C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1680495506.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan"]=np.round(df["Tavan"],2)


In [30]:
df["Tavan Kontrol"]=np.where(((df["Adj Close"]/df["Tavan"].shift(1))==1),1,0)
df=df.drop(["Kademe","Tavan"],axis=1)

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\3090067187.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Tavan Kontrol"]=np.where(((df["Adj Close"]/df["Tavan"].shift(1))==1),1,0)


In [31]:
import pandas_ta as ta

df["SMA 5"]=df["Adj Close"].rolling(5).mean()
df["SMA 10"]=df["Adj Close"].rolling(5).mean()
df["SMA 200"]=df["Adj Close"].rolling(5).mean()

# RSI
df["RSI"] = ta.rsi(df["Adj Close"],14)

# Bağıl Hacim
df["Bağıl Hacim"] = df["Volume"] / df["Volume"].rolling(10).mean()



# ADX, DMI, Aroon
adx_data = ta.adx(df["High"], df["Low"], df["Adj Close"], 14)
df["ADX"], df["DMIP"], df["DMIN"] = adx_data.iloc[:, 0], adx_data.iloc[:, 1], adx_data.iloc[:, 2]




def calculate_stoch_rsi(data, rsi_period=14, stochastic_period=14, k_period=3, d_period=3):
    rsi = ta.rsi(data["Adj Close"], rsi_period)
    stoch_rsi = (rsi - rsi.rolling(window=stochastic_period).min()) / (rsi.rolling(window=stochastic_period).max() - rsi.rolling(window=stochastic_period).min())
    stoch_rsi_k = stoch_rsi.rolling(window=k_period).mean() * 100
    stoch_rsi_d = stoch_rsi_k.rolling(window=d_period).mean()
    data['StochRSI_%K'] = stoch_rsi_k
    data['StochRSI_%D'] = stoch_rsi_d
    return data

df = calculate_stoch_rsi(df)


def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    data['EMA_12'] = ta.ema(data['Adj Close'], short_period)
    data['EMA_26'] = ta.ema(data['Adj Close'], long_period)
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal_Line'] = ta.ema(data['MACD'], signal_period)
    data['MACD_above_Signal'] = (data['MACD'] > data['Signal_Line']).astype(float)
    return data

df = calculate_macd(df)



# Diğer Göstergeler
df["DD"] = ((df["Adj Close"] / df["High"]) - 1) * 100
df["Range"] = ((df["High"] - df["Low"]) / df["Adj Close"]) * 100

df["MOM"] = ta.mom(df["Adj Close"], 10)



df["Driehaus Momentum"] = ((df["Bağıl Hacim"] > 2) & (df["RSI"] > 50) & (df["MOM"] > 1) & (df["Adj Close"] > df["SMA 5"]) & (df["Adj Close"] > df["SMA 10"]) & (df["Adj Close"] > df["SMA 200"])).astype(int)
df["MACD Yukarı Kesen"] = ((df["Bağıl Hacim"] > 1.5) & (df["MACD"] > 0) & (df["MACD_above_Signal"] == 1)).astype(int)
df["ADXDMI20"] = ((df["Bağıl Hacim"] > 1.3) & (df["ADX"] > 20) & (df["DMIP"] > df["DMIN"])).astype(int)
df["MACD+Stoch"] = ((df["Bağıl Hacim"] > 1.3) & (df["StochRSI_%K"] > df["StochRSI_%D"]) & (df["MACD_above_Signal"] == 1)).astype(int)




In [32]:
df["Return"]=100*((df["Open"]/df["Adj Close"].shift(2))-1)
df["Return"]=df["Return"].shift(-2)
df["Gerçek Return"]=100*((df["Open"]/df["Open"].shift(1))-1)
df["Gerçek Return"]=df["Gerçek Return"].shift(-2)
df["Date"]=pd.to_datetime(df.index)

In [33]:
endeks=endeks["Son.Fyt"]
endeks=endeks.replace(",",".")
endeks=float(endeks)

In [34]:
index=yf.download(tickers="XU100.IS",start="2000-01-01")
df["Endeks"]=index["Adj Close"]
df["Endeks Return"]=100*((df["Endeks"]/df["Endeks"].shift(1))-1)
df["Return Lag"]=((df["Adj Close"]/df["Adj Close"].shift(1))-1)*100
endeks_son=((endeks/index["Adj Close"].iloc[-2])-1)*100
df["Endeks Return"].loc["2024-06-04"]=endeks_son
df

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1314702728.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Endeks Return"].loc["2024-06-04"]=endeks_son
C:\Users\Bor

,High,Low,Open,Adj Close,Volume,Hisse,Tavan Kontrol,SMA 5,SMA 10,SMA 200,...,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,Return,Gerçek Return,Date,Endeks,Endeks Return,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-15,27.50,27.50,27.50,27.50,7.210885e+06,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,12.727273,2.513228,2023-06-15,5495.500000,NaN,NaN
2023-06-16,30.24,30.24,30.24,30.24,3.536565e+07,A1CAP.IS,1,NaN,NaN,NaN,...,0,0,0,0,-10.846561,-13.032258,2023-06-16,5475.500000,-0.363934,9.963636
2023-06-20,31.10,29.94,31.00,29.94,2.414492e+08,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,-14.428858,-4.970326,2023-06-20,5251.600098,-4.089122,-0.992063
2023-06-21,26.96,26.96,26.96,26.96,5.789431e+07,A1CAP.IS,0,NaN,NaN,NaN,...,0,0,0,0,-3.041543,2.029664,2023-06-21,5186.799805,-1.233915,-9.953240
2023-06-22,27.62,25.50,25.62,25.94,1.865039e+09,A1CAP.IS,0,28.116,28.116,28.116,...,0,0,0,0,-0.154202,-0.918133,2023-06-22,5428.299805,4.656050,-3.783383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-29,7.61,7.22,7.22,7.39,3.683576e+07,ZRGYO.IS,0,7.220,7.220,7.220,...,0,0,1,0,7.036536,4.216074,2024-05-29,10479.200195,-1.857174,2.354571
2024-05-30,7.97,7.22,7.59,7.89,9.409127e+07,ZRGYO.IS,0,7.354,7.354,7.354,...,0,1,1,1,3.422053,3.160556,2024-05-30,10481.299805,0.020036,6.765900
2024-05-31,8.20,7.53,7.91,8.16,1.656586e+08,ZRGYO.IS,0,7.566,7.566,7.566,...,0,1,1,1,-1.470588,-1.470588,2024-05-31,10400.500000,-0.770895,3.422053


In [35]:
bist100=pd.read_excel("./excel/bist100.xlsx")
bist100=bist100["Kod"].values
for i in range(len(bist100)):
    bist100[i]=bist100[i]+".IS"

In [36]:

yasak=["YYAPI","EMNIS","TETMT","RODRG","BRKO","AYES","EUKYO","YGYO","BALAT","SONME","SNKRN","KSTUR","DERIM","UZERB","MARKA","MMCAS","YAYLA"]
for i in range(len(yasak)):
    yasak[i]=yasak[i]+".IS"

In [37]:
df=df.dropna()

In [38]:
result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])
result=result.droplevel(0)
result

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1343696344.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])


,High,Low,Open,Adj Close,Volume,Hisse,Tavan Kontrol,SMA 5,SMA 10,SMA 200,...,Driehaus Momentum,MACD Yukarı Kesen,ADXDMI20,MACD+Stoch,Return,Gerçek Return,Date,Endeks,Endeks Return,Return Lag
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-08,46.14,42.02,45.80,42.70,5.898352e+08,A1CAP.IS,0,40.908,40.908,40.908,...,0,0,0,0,1.733021,1.023256,2023-08-08,7412.200195,-0.775089,-3.655235
2023-08-09,45.16,42.50,43.00,42.80,4.983311e+08,A1CAP.IS,0,42.112,42.112,42.112,...,0,0,0,0,-4.719626,-6.123389,2023-08-09,7600.700195,2.543105,0.234192
2023-08-10,44.90,39.50,43.44,40.68,4.606404e+08,A1CAP.IS,0,42.160,42.160,42.160,...,0,0,0,0,3.392330,3.138794,2023-08-10,7441.500000,-2.094546,-4.953271
2023-08-11,42.02,39.00,40.78,41.50,4.842361e+08,A1CAP.IS,0,42.400,42.400,42.400,...,0,0,0,0,-0.240964,-1.569187,2023-08-11,7714.399902,3.667270,2.015733
2023-08-14,43.26,40.94,42.06,41.10,4.254512e+08,A1CAP.IS,0,41.756,41.756,41.756,...,0,0,0,0,4.525547,3.768116,2023-08-14,7737.399902,0.298144,-0.963855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27,7.47,7.11,7.11,7.17,1.954102e+07,ZRGYO.IS,0,7.346,7.346,7.346,...,0,0,0,0,0.697350,0.697350,2024-05-27,10657.000000,-0.184516,0.985915
2024-05-28,7.27,7.04,7.17,7.22,9.543981e+06,ZRGYO.IS,0,7.256,7.256,7.256,...,0,0,0,0,5.124654,5.124654,2024-05-28,10677.500000,0.192362,0.697350
2024-05-29,7.61,7.22,7.22,7.39,3.683576e+07,ZRGYO.IS,0,7.220,7.220,7.220,...,0,0,1,0,7.036536,4.216074,2024-05-29,10479.200195,-1.857174,2.354571


In [39]:
result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])
result=result.droplevel(0)
from sklearn.preprocessing import MinMaxScaler
result=result[result["Hisse"].isin(bist100)]
features=["Endeks Return","Return Lag","Range","RSI","Volume","Bağıl Hacim","DD","MACD+Stoch","Driehaus Momentum","ADXDMI20","MACD Yukarı Kesen","High","Low","Open","Adj Close"]
scaler1=MinMaxScaler(feature_range=(0,1))
result[features]=scaler1.fit_transform(result[features])
scaler2=MinMaxScaler(feature_range=(0,1))
result["Return"]=scaler2.fit_transform(np.array(result["Return"]).reshape(-1,1))
test=result.loc["2024-06-04"]
X_test=test[["Endeks Return","Return Lag","Range","RSI","Volume","Bağıl Hacim","DD","MACD+Stoch","Driehaus Momentum","ADXDMI20","MACD Yukarı Kesen","High","Low","Open","Adj Close"]]



C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1614636156.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('Hisse').apply(lambda x: x.iloc[2:])


In [40]:
params={'iterations': 571,
 'depth': 5,
 'learning_rate': 0.05514261151828995,
 'random_strength': 4,
 'bagging_temperature': 0.9572252022114495,
 'border_count': 59,
 'l2_leaf_reg': 20}
model=CatBoostRegressor(**params,task_type="GPU",random_seed=123)
model.load_model("./model/bist100scaling.bin")

In [41]:
tahminler=pd.DataFrame(scaler2.inverse_transform(np.array(test["Return"]).reshape(-1,1)))
tahminler=tahminler.set_index(X_test.index)
tahminler["Tahmin"]=scaler2.inverse_transform(np.array(model.predict(X_test)).reshape(-1,1))
tahminler.columns=["Gerçek","Tahmin"]
tahminler["Hisse"]=test["Hisse"]
tahminler["Tavan"]=test["Tavan Kontrol"]
tahminler=tahminler[tahminler["Tavan"]==0]
tahminler=tahminler.sort_values(by="Tahmin",ascending=False).head(5)
tahminler

,Gerçek,Tahmin,Hisse,Tavan
Date,,,,
2024-06-04,-90.000000,-1.433803,TTKOM.IS,0
2024-06-04,-96.720099,-1.519949,TURSG.IS,0
2024-06-04,-95.681818,-1.542926,KRDMD.IS,0
2024-06-04,-95.797820,-1.588087,TAVHL.IS,0
2024-06-04,-29.078014,-1.679636,TABGD.IS,0


In [42]:
indexler=df.index.unique()
tahminler=tahminler.set_index(pd.to_datetime(["2024-06-03","2024-06-03","2024-06-03","2024-06-03","2024-06-03"]))

In [43]:
window_size=10
vars=[]
tahminler["Var"]=0
for index, group in tahminler.groupby(level=0):
    hisseler=group["Hisse"]
    for hisse in hisseler:
        try:
           window_data=df[df["Hisse"]==hisse]["Adj Close"].loc[indexler[-500]:].loc[:index].iloc[-window_size:]
           getiriler=window_data.pct_change(window_size-1)*100
           getiri=getiriler.iloc[-1]
           std=window_data.std()
           var=getiri/std
           vars.append(var)
           tahminler.loc[(tahminler["Hisse"]==hisse) & (tahminler.index==index), "Var"] = var
        except Exception as e:
            print(e)
tahminler

C:\Users\Bora\AppData\Local\Temp\ipykernel_9252\1541433454.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.566195719264111' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tahminler.loc[(tahminler["Hisse"]==hisse) & (tahminler.index==index), "Var"] = var


,Gerçek,Tahmin,Hisse,Tavan,Var
2024-06-03,-90.000000,-1.433803,TTKOM.IS,0,5.566196
2024-06-03,-96.720099,-1.519949,TURSG.IS,0,3.026045
2024-06-03,-95.681818,-1.542926,KRDMD.IS,0,8.407792
2024-06-03,-95.797820,-1.588087,TAVHL.IS,0,1.461363
2024-06-03,-29.078014,-1.679636,TABGD.IS,0,0.079225


In [44]:
orj_test=scaler1.inverse_transform(X_test)
orj_test=pd.DataFrame(orj_test,columns=X_test.columns)
orj_test["Hisse"]=test[test["Hisse"].isin(bist100)]["Hisse"].values
orj_test

,Endeks Return,Return Lag,Range,RSI,Volume,Bağıl Hacim,DD,MACD+Stoch,Driehaus Momentum,ADXDMI20,MACD Yukarı Kesen,High,Low,Open,Adj Close,Hisse
0,-1.821005,0.000000,4.252336,67.069735,5.674045e+08,1.036576,-3.123585,0.0,0.0,0.0,0.0,220.90,211.80,215.00,214.00,AEFES.IS
1,-1.821005,-6.400438,7.656341,21.643251,1.051070e+09,0.412710,-7.111835,0.0,0.0,0.0,0.0,18.42,17.11,18.40,17.11,AGROT.IS
2,-1.821005,-0.214592,1.505376,42.522566,8.920453e+07,0.477073,-1.133948,0.0,0.0,0.0,0.0,14.11,13.90,13.95,13.95,AHGAZ.IS
3,-1.821005,-4.940120,6.141732,52.695161,3.627183e+09,0.715826,-5.786350,0.0,0.0,0.0,0.0,67.40,63.50,66.80,63.50,AKBNK.IS
4,-1.821005,-0.658858,3.610906,33.662148,8.973705e+07,0.709470,-2.233429,0.0,0.0,0.0,0.0,138.80,133.90,136.70,135.70,AKCNS.IS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.821005,0.639535,2.657423,42.927302,2.946857e+08,0.582331,-1.142204,0.0,0.0,0.0,0.0,87.55,85.25,86.20,86.55,VESTL.IS
96,-1.821005,-3.870626,4.467733,38.696937,1.784237e+08,0.540596,-4.276663,0.0,0.0,0.0,0.0,189.40,181.30,188.40,181.30,YEOTK.IS
97,-1.821005,-4.501217,5.796178,44.089150,4.320840e+09,0.634392,-5.307600,0.0,0.0,0.0,0.0,33.16,31.34,32.88,31.40,YKBNK.IS
98,-1.821005,-0.810373,2.450980,33.622694,8.988631e+07,0.738930,-2.080000,0.0,0.0,0.0,0.0,12.50,12.20,12.35,12.24,YYLGD.IS


In [51]:
bakiye=4051.02
veri=tahminler[tahminler["Var"]>0]
veri=veri.head(3)
veri=veri.sort_values(by="Hisse",ascending=True)
veri['Ağırlık'] = veri['Var'] / veri.groupby(level=0)['Var'].transform('sum')
veri["Fiyat"]=orj_test[orj_test["Hisse"].isin(veri["Hisse"])]["Adj Close"].values
veri["Para"]=bakiye*veri["Ağırlık"]
veri["Lot"]=veri["Para"]//(veri["Fiyat"]*1.1)
veri

,Gerçek,Tahmin,Hisse,Tavan,Var,Ağırlık,Fiyat,Para,Lot
2024-06-03,-95.681818,-1.542926,KRDMD.IS,0,8.407792,0.494575,30.8,2003.533414,59.0
2024-06-03,-90.000000,-1.433803,TTKOM.IS,0,5.566196,0.327423,48.8,1326.395663,24.0
2024-06-03,-96.720099,-1.519949,TURSG.IS,0,3.026045,0.178002,81.1,721.090923,8.0
